In [1]:
%load_ext autoreload
%autoreload 2

import csv
import ipywidgets as widgets
import numpy as np
import os
import pandas as pd
import time

from benchmarks import gsa_svm_fitness
from src.entities import GSA

from IPython.display import display
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from typing import Mapping, Tuple, Union
from ucimlrepo import fetch_ucirepo

In [2]:
uci_datasets = ["(Small) Breast Cancer Wisconsin (Diagnostic)",
                "(Mid) Spambase",
                "(Large) Mushroom"]

widget_opt = widgets.Dropdown(
    options=uci_datasets,
    description='Dataset: '
)

display(widget_opt)

Dropdown(description='Dataset: ', options=('(Small) Breast Cancer Wisconsin (Diagnostic)', '(Mid) Spambase', '…

In [3]:
def fetch_categorical_dataset(_id: int) -> Tuple[pd.DataFrame, pd.Series]:
    data = fetch_ucirepo(id=_id)
    X = data.data.features
    y = data.data.targets
    encoder = LabelEncoder()
    for col in X.columns:
        X.loc[:, col] = encoder.fit_transform(X[col])
    return X, y

if widget_opt.value == "(Small) Breast Cancer Wisconsin (Diagnostic)":
    data = fetch_ucirepo(id=15)
    X = data.data.features
    y = data.data.targets
    X = X.fillna(value=0)
elif widget_opt.value == "(Mid) Spambase":
    X, y = fetch_categorical_dataset(_id=73)
elif widget_opt.value == "(Large) Mushroom":
    X, y = fetch_categorical_dataset(_id=24)

In [4]:
class UCI:
    """
    Class to handle UCI datasets
    
    Args:
        X (pd.DataFrame): Features
        y (pd.Series): Target
    """
    def __init__(self,
                 X: pd.DataFrame,
                 y: pd.Series,
                 boundaries: Mapping[str, Tuple[Tuple[float, float], ...]]
                 ) -> None:
        """
        Constructor
        
        Args:
            X (pd.DataFrame): Features
            y (pd.Series): Target
        """
        self.X = X
        self.y = y
        self.boundaries = boundaries

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.X)
        
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.20, random_state=5)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

    
    def get_fitness(self,
                    solution: Mapping[str, np.ndarray],
                    data: Union[None, Tuple[np.ndarray, np.ndarray]] = None,
                    ) -> Tuple[float, float]:
        """
        Get fitness of a solution
        
        Args:
            solution (Mapping[str, np.ndarray]): Solution to evaluate
            data (Union[None, Tuple[np.ndarray, np.ndarray]], optional): Data to evaluate the solution. Defaults to None.
        
        Returns:
            Tuple[float, float]: Fitness and accuracy of the solution    
        """
        if data is None:
            X_scaled = self.X_train
            y_data = self.y_train
        else:
            X_scaled, y_data = data
            
        gamma, C = solution['real']
        gamma /= 1000
        C /= 1000
        X_scaled_filtered = X_scaled[:, solution['discrete'].astype(int) == 1]
        svc_model = SVC(gamma=gamma, C=C, kernel="rbf", verbose=False)
        svc_model.fit(X_scaled_filtered, np.ravel(y_data))
        y_predict = svc_model.predict(X_scaled_filtered)
        conf_matrix = confusion_matrix(y_data, y_predict)
        accuracy = accuracy_score(y_data, y_predict) * 100
        
        return gsa_svm_fitness(accuracy=accuracy, solution=solution)

    def is_feasible(self, solution: Mapping[str, np.ndarray]) -> bool:
        """
        Check if a solution is feasable

        Args:
            solution (Mapping[str, np.ndarray]): Solution to evaluate

        Returns:
            bool: True if the solution is feasable, False otherwise
        """
        real_values = solution['real']
        discrete_values = solution['discrete']

        for i, (min_val, max_val) in enumerate(self.boundaries['real']):
            if real_values[i] < min_val or real_values[i] > max_val:
                return False

        for i, (min_val, max_val) in enumerate(self.boundaries['discrete']):
            if discrete_values[i] < min_val or discrete_values[i] > max_val:
                return False

        return True

boundaries = {'real': [(1, 100_000), (1, 100_000)], 'discrete': [(0, 1) for _ in range(len(X.columns))]}
uci = UCI(X, y, boundaries)

In [5]:
uci.is_feasible({'real': np.array([1.91833448, 34.69824839]),
                 'discrete': np.array([1, 1, 0, 1, 1, 1, 0, 1, 0])})

True

In [6]:
# Select number of repetitions for each experiment. 
runs = 10

# Select chaotic constant
chaotic_constant = True

# Export results ?
export = True

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

# Check if it works at least once
atLeastOneIteration = False

population_size = 5 
iterations = 20


# CSV Header for the convergence
CnvgHeader = ["Sol_Iter"+str(l+1) for l in range(iterations)]

best_solution_history = []
accuracy_history = []
gamma_history = []
c_history = []
n_features_history = []

uci = UCI(X, y, boundaries)

for k in range(runs):
    gsa_algo = GSA(objective_function = uci.get_fitness,
                   is_feasible=uci.is_feasible,
                   r_dim=2,
                   d_dim=len(X.columns),
                   boundaries=uci.boundaries,
                   repair_solution=False)
    
    gsa_algo.optimize(population_size=population_size,
                      iters=iterations,
                      chaotic_constant=True)
    
    print(gsa_algo.solution_history[-1])
    fitness, accuracy = uci.get_fitness(solution=gsa_algo.solution_history[-1],
                                        data=(uci.X_test, uci.y_test))
    
    print("Test accuracy: ", accuracy, " - Fitness: ", fitness)
    
    best_solution_history.append(gsa_algo.solution_history[-1])
    accuracy_history.append(accuracy)
    gamma_history.append(gsa_algo.solution_history[-1]['real'][0] / 1000)
    c_history.append(gsa_algo.solution_history[-1]['real'][1] / 1000)
    n_features_history.append(np.sum(gsa_algo.solution_history[-1]['discrete']))
    
    if export:
        with open(ExportToFile, 'a') as out:
            writer = csv.writer(out, delimiter=',')
            if not atLeastOneIteration:  # just one time to write the header of the CSV file
                header = np.concatenate(
                    [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                writer.writerow(header)
            a = np.concatenate(
                [[gsa_algo.objective_function_name,
                  gsa_algo.start_time,
                  gsa_algo.end_time,
                  gsa_algo.execution_time],
                  gsa_algo.convergence])
            writer.writerow(a)
        out.close()
    atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Failed to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "get_fitness"
['At iteration 1 the best fitness is 90.39554760484995']
['At iteration 2 the best fitness is 91.04353011329758']
['At iteration 3 the best fitness is 91.47286821705427']
['At iteration 4 the best fitness is 91.47286821705427']
['At iteration 5 the best fitness is 91.47286821705427']
['At iteration 6 the best fitness is 91.47286821705427']
['At iteration 7 the best fitness is 91.47286821705427']
['At iteration 8 the best fitness is 91.47286821705427']
['At iteration 9 the best fitness is 91.47286821705427']
['At iteration 10 the best fitness is 91.47286821705427']
['At iteration 11 the best fitness is 91.47286821705427']
['At iteration 12 the best fitness is 91.61598091830652']
['At iteration 13 the best fitness is 91.61598091830652']
['At iteration 14 the best fitness is 91.61598091830652']
['At iteration 15 the best fitness is 91.61598091830652']
['At iteration 16 the best fitness is 91.61598091830652']
['At iteration 17 the best fitness is 91.6159809

In [7]:
best_solution_history

[{'real': array([78973.33760299, 56089.98811876]),
  'discrete': array([1, 0, 1, 0, 0, 1, 0, 0, 1])},
 {'real': array([97184.81048144, 76744.61063702]),
  'discrete': array([1, 1, 0, 1, 0, 1, 0, 1, 0])},
 {'real': array([36609.43665324, 52598.64351661]),
  'discrete': array([0, 1, 1, 0, 0, 1, 0, 0, 0])},
 {'real': array([29401.01670702, 87678.97562807]),
  'discrete': array([1, 0, 1, 0, 0, 0, 0, 0, 0])},
 {'real': array([98170.99212494, 13371.42006238]),
  'discrete': array([0, 0, 0, 0, 0, 1, 1, 0, 0])},
 {'real': array([17686.48201455, 26140.77855536]),
  'discrete': array([1, 0, 1, 0, 0, 0, 0, 1, 0])},
 {'real': array([15153.01499373, 80366.30004975]),
  'discrete': array([0, 1, 0, 0, 0, 1, 1, 0, 0])},
 {'real': array([17005.00345186, 44487.89465484]),
  'discrete': array([1, 1, 0, 0, 1, 0, 1, 1, 1])},
 {'real': array([22258.93167175, 64437.30333909]),
  'discrete': array([0, 0, 1, 0, 1, 0, 1, 0, 0])},
 {'real': array([54631.72167678, 77231.58143203]),
  'discrete': array([1, 0, 0, 1

In [8]:
accuracy_history

[98.57142857142858,
 100.0,
 99.28571428571429,
 97.14285714285714,
 96.42857142857143,
 99.28571428571429,
 100.0,
 100.0,
 97.14285714285714,
 100.0]

In [9]:
gamma_history

[78.97333760298665,
 97.18481048144302,
 36.60943665323588,
 29.40101670701529,
 98.17099212494213,
 17.686482014551483,
 15.153014993732802,
 17.005003451860215,
 22.258931671745728,
 54.63172167677966]

In [10]:
c_history

[56.08998811875777,
 76.74461063702333,
 52.598643516614125,
 87.67897562806765,
 13.371420062380839,
 26.140778555357624,
 80.36630004975137,
 44.4878946548438,
 64.43730333908614,
 77.23158143203186]

In [11]:
mean_accuracy = np.mean(accuracy_history)
std_accuracy = np.std(accuracy_history)

print(f"Mean accuracy: {mean_accuracy} +/- {std_accuracy}")

mean_gamma = np.mean(gamma_history)
std_gamma = np.std(gamma_history)

print(f"Mean gamma: {mean_gamma} +/- {std_gamma}")

mean_c = np.mean(c_history)
std_c = np.std(c_history)

print(f"Mean C: {mean_c} +/- {std_c}")

mean_n_features = np.mean(n_features_history)
std_n_features = np.std(n_features_history)

print(f"Mean n_features: {mean_n_features} +/- {std_n_features}")

Mean accuracy: 98.78571428571429 +/- 1.3190132366156715
Mean gamma: 46.70747473782929 +/- 31.653891563715053
Mean C: 57.91474959939145 +/- 23.164181938807417
Mean n_features: 3.6 +/- 1.2806248474865698


                /\ 
               /  \
                || 
                
        Nuestros resultados
        
Dataset: Mushroom
Instances: 8124 (Train / Test: 80% / 20%)
Features: 22

Resultados autores:

| Metric | Value             |
| --- |-------------------|
| Accuracy | 98.06 +/- 0.78    |
| Gamma | 0.0067 +/- 0.0144 |
| C | 47.35 +/- 27.57   |
| n_features | 3 +/- 2.16        |

                /\ 
               /  \
                || 
                
        Nuestros resultados
        
Dataset: Breast Cancer Wisconsin (Diagnostic) Data Set

Resultados autores:

| Metric | Value             |
| --- |-------------------|
| Accuracy | 99.54 +/- 0.25    |
| Gamma | 0.0685 +/- 0.1293 |
| C | 40.30 +/- 22.37   |
| n_features | 2 +/- 1           |

In [8]:
solution= {'real': np.array([0.91833448, 34.69824839]),
           'discrete': np.array([1, 1, 0, 1, 1, 1, 0, 1, 0])}

fitness, accuracy = uci.get_fitness(solution=solution,
                                    data=(uci.X_test_scaled, uci.y_test))

print("Test accuracy: ", accuracy, " - Fitness: ", fitness)

Test accuracy:  100.0  - Fitness:  0.2666666666666666
